In [1]:
%matplotlib inline

In [107]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import sqlite3

# Tabular Data Overview
----

Working with tabular data in Python generally means working with the [pandas](https://pandas.pydata.org/) library. For the purposes of this discussion, pandas provides two useful data structures: A Series object (analogous to a dictionary), and a DataFrame object, which is analogous to an Excel spreadsheet, from which columns or rows can be extracted as Series objects.

DataFrames can be read in from delimited (comma, tab, etc.) text files, Excel files, database tables, or created from scratch. DataFrame objects provide a number of useful methods for data exploration and analysis including:

 - computation
 - statistics
 - plotting


## Contents
----
- [Reading ComCat Catalogs](#Reading-ComCat-Catalogs)
- [Reading PAGER Exposures](#Reading-PAGER-Exposures)
- [Data with Other Delimiters](#Data-with-Other-Delimiters)
- [Reading Excel Files](#Reading-Excel-Files)
- [Reading Database Tables](#Reading-Database-Tables)

## Reading Data from ComCat

This is a standard libcomcat catalog data set (obtained via getcsv) containing all M5.5+ events globally from 1970 through mid-2020. Reading it into a pandas dataframe should be fairly straightforward.

In [3]:
eventfile = '../data/mag55_1970_2020.csv'

In [5]:
mag55_events = pd.read_csv(eventfile)
print(f'Table has {len(mag55_events)} events')
mag55_events.head() # show first 5 rows of the dataframe

Table has 23268 events


,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance
0,iscgem799588,1970-01-01 17:11:00.000,"Kermadec Islands, New Zealand",-29.400,-177.169,35.0,5.6,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,482
1,iscgem799712,1970-01-04 17:00:41.000,"Yunnan, China",24.185,102.543,11.3,7.1,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,776
2,iscgem799745,1970-01-05 11:49:10.000,"Yunnan, China",23.984,102.732,15.0,5.9,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,536
3,iscgem799772,1970-01-06 05:35:54.000,D'Entrecasteaux Islands region,-9.583,151.493,15.0,6.3,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,611
4,iscgem799824,1970-01-07 07:56:14.000,"east of Guadeloupe, Leeward Islands",15.785,-59.808,36.7,6.0,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,554


Let's try to get the range of earthquake times, formatted like "Mar 30, 1971".

In [6]:
# get time range of earthquakes
tmin = mag55_events['time'].min().strftime('%b %d, %Y')
tmax = mag55_events['time'].max().strftime('%b %d, %Y')
print(f'Earthquakes from {tmin} to {tmax}')


AttributeError: 'str' object has no attribute 'strftime'

This error means that the data type of the *time* column is not a datetime type, which we might have expected, but instead just a series of strings. We can fix this by adding an optional parameter to read_csv, telling it which columns to try to interpret as date/time fields.

In [7]:
mag55_events = pd.read_csv(eventfile, parse_dates=['time'])
mag55_events.head()

,id,time,location,latitude,longitude,depth,magnitude,alert,url,eventtype,significance
0,iscgem799588,1970-01-01 17:11:00,"Kermadec Islands, New Zealand",-29.400,-177.169,35.0,5.6,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,482
1,iscgem799712,1970-01-04 17:00:41,"Yunnan, China",24.185,102.543,11.3,7.1,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,776
2,iscgem799745,1970-01-05 11:49:10,"Yunnan, China",23.984,102.732,15.0,5.9,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,536
3,iscgem799772,1970-01-06 05:35:54,D'Entrecasteaux Islands region,-9.583,151.493,15.0,6.3,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,611
4,iscgem799824,1970-01-07 07:56:14,"east of Guadeloupe, Leeward Islands",15.785,-59.808,36.7,6.0,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,earthquake,554


Now we can ask the dataframe questions about it's contents.

What are the ranges of dates and magnitudes?

In [8]:
tmin = mag55_events['time'].min().strftime('%b %d, %Y')
tmax = mag55_events['time'].max().strftime('%b %d, %Y')
print(f'Earthquakes from {tmin} to {tmax}')
# What is the range of magnitudes?
magmin = mag55_events['magnitude'].min()
magmax = mag55_events['magnitude'].max()
print(f'Magnitudes: M{magmin:.1f} to M{magmax:.1f}')

Earthquakes from Jan 01, 1970 to Jul 26, 2020
Magnitudes: M5.5 to M9.1


## Reading PAGER Exposures

Another libcomcat tool, *getpager*, allows you to retrieve PAGER population exposure to shaking and fatality estimates*. 

Let's use a shell *head* command to look at the beginning of one of these files.

*PAGER results are only available from 2013, and fatality estimates only from May 2017.

In [50]:
!head ../data/pager_exposures.csv

#
#This data represents the results of running the PAGER exposure and loss
#algorithms on the output from ShakeMap.
#
#Notes: "Total" in the country column indicates that the results in that row are
#the sum of exposures/losses for all affected countries.
#
#"predicted_fatalities" and "predicted_dollars" are the results of applying loss
#models to the exposure data - note that these values are not guaranteed to
#match the actual losses from the earthquake.


Note that the file begins with comment lines that begin with *#* characters. Fortunately, pandas allows us to skip these comment lines, as below. Note that there is again a "time" column.

In [52]:
pager_file = '../data/pager_exposures.csv'
pager_results = pd.read_csv(pager_file, comment='#', parse_dates=['time'])
pager_results.head()

,id,location,time,latitude,longitude,depth,magnitude,country,pager_version,mmi1,...,mmi5,mmi6,mmi7,mmi8,mmi9,mmi10,predicted_fatalities,fatality_sigma,predicted_dollars,dollars_sigma
0,us10008qsb,"62km NE of Port-Olry, Vanuatu",2017-05-09 13:52:10.940,-14.5884,167.3767,169.0,6.8,Total,1,0,...,663,0,0,0,0,0,0,2.0,0,2.0
1,us10008rb6,"95km ENE of Visokoi Island, South Georgia and ...",2017-05-10 23:23:36.830,-56.4140,-25.7432,15.0,6.5,Total,1,0,...,0,0,0,0,0,0,0,1.0,0,2.0
2,us10008w1z,"28km WNW of Kasiguncu, Indonesia",2017-05-29 14:35:21.510,-1.2923,120.4313,12.0,6.6,Total,1,10,...,548586,91583,23084,0,0,0,0,2.0,166266,410465.0
3,us20009jd6,"200km NW of Attu Station, Alaska",2017-06-02 22:24:47.440,54.0312,170.9196,5.0,6.8,Total,1,0,...,0,0,0,0,0,0,0,1.0,0,1.0
4,us20009jd6,"200km NW of Attu Station, Alaska",2017-06-02 22:24:47.440,54.0312,170.9196,5.0,6.8,Total,2,0,...,0,0,0,0,0,0,0,1.0,0,1.0


## Data with Other Delimiters

The pandas read_csv() method is not limited to reading text files with columns separated by commas. The delimiter can be just about anything - tabs are very common, as are pipes ("|"), as we see below. This data set is extracted from the Hydra QA database, and represents NEIC analyst data collected on earthquake impacts.

In [54]:
!head -5 ../data/hydra_impact_comments.txt

In [79]:
impact_file = '../data/hydra_impact_comments.txt'
impacts = pd.read_csv(impact_file, delimiter='|', skipfooter=2, engine='python')
impacts.head()

,huidevent,ot,mag,spassportcmd,spassportentry
0,C0004KTB,2011/06/29 23:16:39,4.925,PubFlagsAddImpact,"Injuries Shaking Exact 7 0 ""http://www.huffin..."
1,C0005IDZ,2011/08/23 05:46:18,5.269,PubFlagsAddImpact,"Damaged Shaking Few 0 0 """" 1317972203 ""(VII) ..."
2,C0005WX9,2011/09/19 18:33:59,5.801,PubFlagsAddImpact,"Deaths Shaking Exact 1 0 ""AP from Guatemala C..."
3,C0005WX9,2011/09/19 18:33:59,5.801,PubFlagsAddImpact,"Injuries Shaking Some 0 0 ""AP from Guatemala ..."
4,C0007NLY,2012/01/19 12:35:52,5.105,PubFlagsAddImpact,"Injuries Unknown AtLeast 200 0 """" 1327037928 """""


The first thing that will trip us up here is that there are spaces around the column names, like this:

In [81]:
impacts.columns

Index([' huidevent ', '         ot          ', '  mag  ',
       '   spassportcmd    ',
       '                                                                                                                                                                                                                                                 spassportentry                                                                                                                                                                                                                                                 '],
      dtype='object')

Fortunately, pandas provides us with a trick for removing those spaces. 

TODO: Interpret this command for the user!

In [82]:
impacts.columns = impacts.columns.str.replace(' ', '')
print(impacts.columns)
impacts.head()

Index(['huidevent', 'ot', 'mag', 'spassportcmd', 'spassportentry'], dtype='object')


,huidevent,ot,mag,spassportcmd,spassportentry
0,C0004KTB,2011/06/29 23:16:39,4.925,PubFlagsAddImpact,"Injuries Shaking Exact 7 0 ""http://www.huffin..."
1,C0005IDZ,2011/08/23 05:46:18,5.269,PubFlagsAddImpact,"Damaged Shaking Few 0 0 """" 1317972203 ""(VII) ..."
2,C0005WX9,2011/09/19 18:33:59,5.801,PubFlagsAddImpact,"Deaths Shaking Exact 1 0 ""AP from Guatemala C..."
3,C0005WX9,2011/09/19 18:33:59,5.801,PubFlagsAddImpact,"Injuries Shaking Some 0 0 ""AP from Guatemala ..."
4,C0007NLY,2012/01/19 12:35:52,5.105,PubFlagsAddImpact,"Injuries Unknown AtLeast 200 0 """" 1327037928 """""


Unfortunately, our job is not really done - the information we're interested in is contained in the "spassportentry" column, and *it* is tab-separated. To solve this, we'll do the following:

 - loop over each row of the "spassportentry" column
 - parse that tab separated string into a list of strings
 - use a "list comprehension" to clean up those strings
 - create a row dictionary from the strings
 - add that row to a list of rows
 - construct a dataframe from list of row dictionaries

In [106]:
IMPACT_COLUMNS = ['LossExtent',
                  'EffectType',
                  'LossQuantifier',
                  'LossValue',
                  'Location',
                  'CollectionSource',
                  'database_id',
                  'comment']
rows = []
for idx, value in impacts['spassportentry'].iteritems():
    parts = value.strip().split()
    parts = [p.replace('"', '') for p in parts]
    row = dict(zip(IMPACT_COLUMNS,parts))
    rows.append(row)
impact_frame = pd.DataFrame(rows)
impact_frame.head()

,LossExtent,EffectType,LossQuantifier,LossValue,Location,CollectionSource,database_id,comment
0,Injuries,Shaking,Exact,7,0,http://www.huffingtonpost.com/2011/06/29/japan...,1309305961,
1,Damaged,Shaking,Few,0,0,,1317972203,(VII)
2,Deaths,Shaking,Exact,1,0,AP,from,Guatemala
3,Injuries,Shaking,Some,0,0,AP,from,Guatemala
4,Injuries,Unknown,AtLeast,200,0,,1327037928,


## Reading Database Tables

You can also read database tables or results from queries into pandas DataFrames. By default, the file-based *sqlite* database is supported. Other databases are supported through an *Object Relational Mapper (ORM)* library called SQLAlchemy, but this is beyond the scope of this discussion.

You can do simple queries to get all the contents of one table...

In [123]:
conn = sqlite3.connect('../data/flatdb.sqlite')
networks = pd.read_sql('select * from network', conn)
networks

,id,code,desc
0,1,BK,BK
1,2,CI,CI
2,3,GS,GS
3,4,NC,NC
4,5,NN,NN
5,6,NP,NP
6,7,AK,AK


Or more complex queries to join fields from multiple tables.

In [124]:
query = '''SELECT network.code as network, station.code as station, station.description, 
station.latitude, station.longitude, station.elevation,
structure.description as structure_type
FROM network
INNER JOIN station ON station.network_id
INNER JOIN structure ON station.structure_id = structure.id
WHERE network.code = 'CI'
'''
stations = pd.read_sql(query, conn)
stations.head()

,network,station,description,latitude,longitude,elevation,structure_type
0,CI,OVRO,"Owens Valley Radio Observatory, Big Pine, CA",37.230660,-118.283000,1216.0,Sensors buried/set in ground
1,CI,MLAC,Mammoth,37.630192,-118.836052,2162.0,Sensors buried/set in ground
2,CI,MCA04,"MCA04, Monte Cristo Range, NV",38.076720,-117.708500,1661.5,Sensors buried/set in ground
3,CI,MBS1,Big Spring,37.761024,-118.944542,2025.0,Sensors buried/set in ground
4,CI,MCB,Casa Benchmark,37.644394,-118.896835,2391.0,Sensors buried/set in ground


## Reading Excel Files

This data set is one I constructed from the [NOAA Significant Earthquake Database](https://www.ngdc.noaa.gov/hazel/view/hazards/earthquake/search) for the PAGER project. Earthquake and tsunami information was extracted from that website, and then (when possible) some basic ComCat information for those events was extracted.

In [126]:
noaa_file = '../data/noaa_events_2000_2020.xlsx'

In [134]:
noaa_losses = pd.read_excel(noaa_file)
# There are a lot of columns in this data set, 
# so let's pick a representative few and show those
NOAA_COLUMNS = ['Source Network', 'ID', 
           'Time', 'Magnitude', 
           'NOAA Time', 'NOAA Magnitude',
          'EffectType','LossType','LossExtent','LossValue']
noaa_losses[NOAA_COLUMNS].head(10)

,Source Network,ID,Time,Magnitude,NOAA Time,NOAA Magnitude,EffectType,LossType,LossExtent,LossValue
0,us,p0009m1j,2000-01-11 23:43:56.450,5.1,2000-01-11,5.1,tsunami,buildings,damaged,8800
1,us,p0009m1j,2000-01-11 23:43:56.450,5.1,2000-01-11,5.1,tsunami,buildings,destroyed,3600
2,us,p0009m1j,2000-01-11 23:43:56.450,5.1,2000-01-11,5.1,tsunami,people,injured,30
3,us,p0009m1j,2000-01-11 23:43:56.450,5.1,2000-01-11,5.1,shaking,people,injured,30
4,NaN,NaN,NaT,NaN,2000-01-14,5.9,tsunami,people,killed,5
5,NaN,NaN,NaT,NaN,2000-01-14,5.9,tsunami,people,injured,2528
6,NaN,NaN,NaT,NaN,2000-01-14,5.9,tsunami,dollars,damaged or destroyed,73500000
7,NaN,NaN,NaT,NaN,2000-01-14,5.9,tsunami,buildings,destroyed,41000
8,NaN,NaN,NaT,NaN,2000-01-14,5.9,shaking,people,injured,2528
9,us,p0009mz6,2000-02-02 22:58:01.550,5.3,2000-02-02,5.3,tsunami,people,killed,1


There are a number of things to note about this dataset:

 - There are multiple rows per earthquake, one for each unique combination of EffectType, LossType, and LossExtent.
 - The Source Network, ID, Time, and Magnitude columns are *ComCat* values, and they could not always be filled in.
 
Let's see how to:

1. Extract only those events that have been linked to ComCat
2. Extract one "metric" - the number of people killed from shaking

In [135]:
found_events = noaa_losses[noaa_losses['ID'].notnull()]
found_events[NOAA_COLUMNS].head()

,Source Network,ID,Time,Magnitude,NOAA Time,NOAA Magnitude,EffectType,LossType,LossExtent,LossValue
0,us,p0009m1j,2000-01-11 23:43:56.450,5.1,2000-01-11,5.1,tsunami,buildings,damaged,8800
1,us,p0009m1j,2000-01-11 23:43:56.450,5.1,2000-01-11,5.1,tsunami,buildings,destroyed,3600
2,us,p0009m1j,2000-01-11 23:43:56.450,5.1,2000-01-11,5.1,tsunami,people,injured,30
3,us,p0009m1j,2000-01-11 23:43:56.450,5.1,2000-01-11,5.1,shaking,people,injured,30
9,us,p0009mz6,2000-02-02 22:58:01.550,5.3,2000-02-02,5.3,tsunami,people,killed,1


In [136]:
shaking = found_events['EffectType'] == 'shaking'
people = found_events['LossType'] == 'people'
killed = found_events['LossExtent'] == 'killed'
shaking_fatalities = found_events[shaking & people & killed]
shaking_fatalities[NOAA_COLUMNS].head()

,Source Network,ID,Time,Magnitude,NOAA Time,NOAA Magnitude,EffectType,LossType,LossExtent,LossValue
26,us,p0009txv,2000-06-04 16:28:26.170,7.9,2000-06-04 00:00:00,7.9,shaking,people,killed,103
33,us,p0009u6r,2000-06-07 23:45:26.680,6.7,2000-06-07 00:00:00,6.7,shaking,people,killed,1
242,us,p000bbb2,2002-09-06 01:21:28.600,6.0,2002-09-06 00:00:00,6.0,shaking,people,killed,2
300,us,p000bnyr,2003-01-22 02:06:34.610,7.6,2003-01-22 00:00:00,7.5,shaking,people,killed,29
309,us,p000bpa1,2003-01-27 05:26:23.040,6.1,2003-01-27 05:26:23,6.1,shaking,people,killed,1


In [ ]:
# this table has source/event id columns split - let's join them into an eventid column, 
# and get rid of the originals
columns = noaa_losses.columns.tolist()
noaa_losses['eventid'] = noaa_losses['Source Network'] + noaa_losses['ID']
noaa_losses.drop(labels=['Source Network', 'ID'], axis=1, inplace=True)
columns.remove('Source Network')
columns.remove('ID')
newcolumns = ['eventid'] + columns
noaa_losses = noaa_losses[newcolumns]